# Understanding Embedding Spaces

## Imports and definitions

In [40]:
import json
import pprint
import pandas as pd
from IPython.display import display
! pip install av
! pip install yt-dlp
import yt_dlp
import random
import av
import av.datasets
from pathlib import Path

## Donwloading the videos

In [4]:
with open('activity_net.v1-3.min.json', 'r') as json_data:
    data = json.load(json_data)

In [5]:
# Extract video information
videos = data.get('database', {})

# Sort videos by the number of moments (annotations)
sorted_videos = sorted(videos.items(), key=lambda x: len(x[1].get('annotations', [])), reverse=True)

# Select the top 10 videos
top_10_videos = sorted_videos[:11]

# Print the top 10 video IDs and their moment counts
for video_id, video_info in top_10_videos:
    print(f"Video ID: {video_id}, Moments: {len(video_info.get('annotations', []))}")

Video ID: o1WPnnvs00I, Moments: 23
Video ID: oGwn4NUeoy8, Moments: 23
Video ID: VEDRmPt_-Ms, Moments: 20
Video ID: qF3EbR8y8go, Moments: 19
Video ID: DLJqhYP-C0k, Moments: 18
Video ID: t6f_O8a4sSg, Moments: 18
Video ID: 6gyD-Mte2ZM, Moments: 18
Video ID: jBvGvVw3R-Q, Moments: 18
Video ID: PJ72Yl0B1rY, Moments: 17
Video ID: QHn9KyE-zZo, Moments: 17
Video ID: 9-yueOtwiL8, Moments: 17


In [32]:
def download_video(url, output_path='downloads/'):
    ydl_opts = {
        'outtmpl': f'{output_path}/%(id)s - %(title)s.%(ext)s',
        'format': 'mp4/bestvideo+bestaudio',
        'merge_output_format': 'mp4'
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([url])

In [34]:
count = 0
for video_id, video_info in top_10_videos:
    video_url = video_info.get('url')
    if video_url and "youtube.com" in video_url and count < 11: #did 11 because one video is private
        try:
            download_video(video_url)
            print(f"Downloaded: {video_id}")
            count += 1
        except Exception as e:
            print(f"Failed to download {video_id}: {e}")
    else:
        print(f"Invalid or unsupported URL for video ID or already gor 10 videos: {video_id}")

[youtube] Extracting URL: https://www.youtube.com/watch?v=o1WPnnvs00I
[youtube] o1WPnnvs00I: Downloading webpage
[youtube] o1WPnnvs00I: Downloading tv client config
[youtube] o1WPnnvs00I: Downloading player e63b9241-main
[youtube] o1WPnnvs00I: Downloading tv player API JSON
[youtube] o1WPnnvs00I: Downloading ios player API JSON
[youtube] o1WPnnvs00I: Downloading m3u8 information
[info] o1WPnnvs00I: Downloading 1 format(s): 18
[download] Destination: downloads\o1WPnnvs00I - Irish March (Fife & Flute) – Wouter Kellerman (Live).mp4
[download] 100% of   10.19MiB in 00:00:05 at 1.76MiB/s     
Downloaded: o1WPnnvs00I
[youtube] Extracting URL: https://www.youtube.com/watch?v=oGwn4NUeoy8
[youtube] oGwn4NUeoy8: Downloading webpage
[youtube] oGwn4NUeoy8: Downloading tv client config
[youtube] oGwn4NUeoy8: Downloading player e63b9241-main
[youtube] oGwn4NUeoy8: Downloading tv player API JSON
[youtube] oGwn4NUeoy8: Downloading ios player API JSON
[youtube] oGwn4NUeoy8: Downloading m3u8 information

ERROR: [youtube] PJ72Yl0B1rY: Private video. Sign in if you've been granted access to this video. Use --cookies-from-browser or --cookies for the authentication. See  https://github.com/yt-dlp/yt-dlp/wiki/FAQ#how-do-i-pass-cookies-to-yt-dlp  for how to manually pass cookies. Also see  https://github.com/yt-dlp/yt-dlp/wiki/Extractors#exporting-youtube-cookies  for tips on effectively exporting YouTube cookies


Failed to download PJ72Yl0B1rY: ERROR: [youtube] PJ72Yl0B1rY: Private video. Sign in if you've been granted access to this video. Use --cookies-from-browser or --cookies for the authentication. See  https://github.com/yt-dlp/yt-dlp/wiki/FAQ#how-do-i-pass-cookies-to-yt-dlp  for how to manually pass cookies. Also see  https://github.com/yt-dlp/yt-dlp/wiki/Extractors#exporting-youtube-cookies  for tips on effectively exporting YouTube cookies
[youtube] Extracting URL: https://www.youtube.com/watch?v=QHn9KyE-zZo
[youtube] QHn9KyE-zZo: Downloading webpage
[youtube] QHn9KyE-zZo: Downloading tv client config
[youtube] QHn9KyE-zZo: Downloading player e63b9241-main
[youtube] QHn9KyE-zZo: Downloading tv player API JSON
[youtube] QHn9KyE-zZo: Downloading ios player API JSON
[youtube] QHn9KyE-zZo: Downloading m3u8 information
[info] QHn9KyE-zZo: Downloading 1 format(s): 18
[download] Destination: downloads\QHn9KyE-zZo - London Slacklining.mp4
[download] 100% of   16.67MiB in 00:00:09 at 1.78MiB/s 

Building a clean list of metadata entries from the downloaded videos

In [36]:
# Preview filenames
print("Downloaded files:")
for f in downloaded_files[:5]:
    print("-", f.name)

# Preview a few video IDs from the metadata
print("\nVideo IDs from JSON:")
for vid in list(video_db.keys())[:5]:
    print("-", vid)


Downloaded files:
- Chinese Brush Painting.mp4
- Irish March (Fife & Flute) – Wouter Kellerman (Live).mp4
- Jessica Ryckewaert plays Congas Drums Mezza Voce, Rhythm Concert.mp4
- London Slacklining.mp4
- LOS MEJORES TIROS DE BOLOS  THE  BEST BOWLING SHOTS.mp4

Video IDs from JSON:
- sJFgo9H6zNo
- V1zhqaGFY2A
- JDg--pjY5gg
- KsFid_YVsn0
- -TmWR_keSfI


In [39]:
# Load the metadata file
with open("activity_net.v1-3.min.json", "r") as f:
    dataset = json.load(f)

video_db = dataset["database"]

# Get the list of renamed video files
download_folder = Path("downloads")
downloaded_files = list(download_folder.glob("*.mp4"))

# Extract the video ID from the new filenames
def extract_video_id(filename):
    return filename.name.split(" - ")[0]  # everything before the first " - "

# Match files to metadata
filtered_metadata = {}

for file in downloaded_files:
    vid = extract_video_id(file)
    if vid in video_db:
        entry = video_db[vid]
        filtered_metadata[vid] = {
            "video_id": vid,
            "title": entry.get("title", ""),
            "description": entry.get("description", ""),
            "url": entry.get("url", ""),
            "duration": entry.get("duration", None),
            "subset": entry.get("subset", ""),
            "annotations": entry.get("annotations", []),
            "file_path": str(file)
        }

# Preview one entry
sample_vid = random.choice(list(filtered_metadata.keys()))
pprint.pprint(filtered_metadata[sample_vid])


{'annotations': [{'label': 'Tumbling',
                  'segment': [15.568780241809671, 21.723879407176288]},
                 {'label': 'Tumbling',
                  'segment': [21.723879407176288, 27.878978572542902]},
                 {'label': 'Tumbling',
                  'segment': [29.32723719968799, 34.75820705148206]},
                 {'label': 'Tumbling',
                  'segment': [34.75820705148206, 39.82711224648986]},
                 {'label': 'Tumbling',
                  'segment': [40.189176903276135, 49.24079332293292]},
                 {'label': 'Tumbling',
                  'segment': [49.60285797971919, 55.757957145085804]},
                 {'label': 'Tumbling',
                  'segment': [57.56828042901716, 64.80957356474259]},
                 {'label': 'Tumbling',
                  'segment': [65.17163822152887, 77.4818365522621]},
                 {'label': 'Tumbling',
                  'segment': [77.4818365522621, 85.80932365834634]},
               

In [42]:
# Convert filtered_metadata to a DataFrame for visualization
df = pd.DataFrame(filtered_metadata.values())

# Display key columns (omit annotations for readability)
preview_df = df[["video_id", "title", "description", "duration", "subset", "url", "file_path"]]

# Show the table in your notebook
display(preview_df)

,video_id,title,description,duration,subset,url,file_path
0,6gyD-Mte2ZM,,,188.245,validation,https://www.youtube.com/watch?v=6gyD-Mte2ZM,downloads\6gyD-Mte2ZM - PBA - Lowest game bowl...
1,9-yueOtwiL8,,,230.690,validation,https://www.youtube.com/watch?v=9-yueOtwiL8,"downloads\9-yueOtwiL8 - Shot-Put, Discus, Jave..."
2,DLJqhYP-C0k,,,186.968,training,https://www.youtube.com/watch?v=DLJqhYP-C0k,downloads\DLJqhYP-C0k - LOS MEJORES TIROS DE B...
3,jBvGvVw3R-Q,,,218.620,training,https://www.youtube.com/watch?v=jBvGvVw3R-Q,downloads\jBvGvVw3R-Q - The King of American W...
4,o1WPnnvs00I,,,229.860,training,https://www.youtube.com/watch?v=o1WPnnvs00I,downloads\o1WPnnvs00I - Irish March (Fife & Fl...
5,oGwn4NUeoy8,,,153.090,training,https://www.youtube.com/watch?v=oGwn4NUeoy8,downloads\oGwn4NUeoy8 - Jessica Ryckewaert pla...
6,qF3EbR8y8go,,,204.100,training,https://www.youtube.com/watch?v=qF3EbR8y8go,downloads\qF3EbR8y8go - Chinese Brush Painting...
7,QHn9KyE-zZo,,,196.279,validation,https://www.youtube.com/watch?v=QHn9KyE-zZo,downloads\QHn9KyE-zZo - London Slacklining.mp4
8,t6f_O8a4sSg,,,218.520,validation,https://www.youtube.com/watch?v=t6f_O8a4sSg,downloads\t6f_O8a4sSg - SKATEBOARDING MADE SIM...
9,VEDRmPt_-Ms,,,232.080,training,https://www.youtube.com/watch?v=VEDRmPt_-Ms,downloads\VEDRmPt_-Ms - Raquel.Pinto.Estrela.M...


Saving filtered metadata to a json file

In [43]:
# Save to JSONL format
with open("filtered_video_metadata.jsonl", "w", encoding="utf-8") as f:
    for item in filtered_metadata.values():
        f.write(json.dumps(item) + "\n")

OpenSearch Part

In [ ]:
import pprint as pp
import requests

host = 'api.novasearch.org'
port = 443

user = '' # Add your user name here.
password = '' # Add your user password here. For testing only. Don't store credentials in code. 
index_name = user

In [48]:
import pprint as pp
from opensearchpy import OpenSearch
from opensearchpy import helpers

# Create the client with SSL/TLS enabled, but hostname verification disabled.
client = OpenSearch(
    hosts = [{'host': host, 'port': port}],
    http_compress = True, # enables gzip compression for request bodies
    http_auth = (user, password),
    use_ssl = True,
    url_prefix = 'opensearch_v2',
    verify_certs = False,
    ssl_assert_hostname = False,
    ssl_show_warn = False
)

if client.indices.exists(index_name):

    resp = client.indices.open(index = index_name)
    print(resp)

    print('\n----------------------------------------------------------------------------------- INDEX SETTINGS')
    settings = client.indices.get_settings(index = index_name)
    pp.pprint(settings)

    print('\n----------------------------------------------------------------------------------- INDEX MAPPINGS')
    mappings = client.indices.get_mapping(index = index_name)
    pp.pprint(mappings)

    print('\n----------------------------------------------------------------------------------- INDEX #DOCs')
    print(client.count(index = index_name))
else:
    print("Index does not exist.")


{'acknowledged': True, 'shards_acknowledged': True}

----------------------------------------------------------------------------------- INDEX SETTINGS
{'user12': {'settings': {'index': {'creation_date': '1742989159969',
                                   'knn': 'true',
                                   'number_of_replicas': '0',
                                   'number_of_shards': '4',
                                   'provided_name': 'user12',
                                   'refresh_interval': '-1',
                                   'replication': {'type': 'DOCUMENT'},
                                   'uuid': 'lG-BTHAbQfGQMNiBJYYfVw',
                                   'version': {'created': '136387927'}}}}}

----------------------------------------------------------------------------------- INDEX MAPPINGS
{'user12': {'mappings': {'dynamic': 'strict',
                         'properties': {'contents': {'analyzer': 'standard',
                                             

In [47]:
import json
from opensearchpy import OpenSearch, helpers

# 1. Connect to OpenSearch (make sure you’ve already defined user, password, etc.)
client = OpenSearch(
    hosts=[{'host': 'api.novasearch.org', 'port': 443}],
    http_compress=True,
    http_auth=(user, password),
    use_ssl=True,
    url_prefix='opensearch_v2',
    verify_certs=False,
    ssl_assert_hostname=False,
    ssl_show_warn=False
)

index_name = "user12"

# 2. Load your saved metadata file
with open("filtered_video_metadata.jsonl", "r", encoding="utf-8") as f:
    lines = f.readlines()

# 3. Build documents for indexing
docs = []
for line in lines:
    item = json.loads(line)
    
    # Create contents field (title + description + maybe labels)
    labels = " ".join(set([ann["label"] for ann in item.get("annotations", [])]))
    contents = f"{item.get('title', '')} {item.get('description', '')} {labels}".strip()

    # Add optional tags based on label(s)
    tags = list(set([ann["label"] for ann in item.get("annotations", [])]))

    doc = {
        "_index": index_name,
        "_id": item["video_id"],
        "_source": {
            "doc_id": item["video_id"],
            "contents": contents,
            "tags": tags,
            "json": item
        }
    }
    docs.append(doc)

# 4. Push to OpenSearch
helpers.bulk(client, docs)
print(f"Indexed {len(docs)} videos into {index_name}")

Indexed 10 videos into user12
